In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import os
import warnings    
warnings.filterwarnings('ignore')
import glob

In [2]:
from IPython.core.display import display, HTML
display(HTML ('<style>.container {width: 80% !important;}</style>'))

In [3]:
# for timing usage: Within 1 min
start_time = datetime.now()

## Configurations

In [4]:
# Required for all
WAMM_DICT_PATH = os.path.abspath('../../data/input_data/WMG WAMM Project Data Dictionary_v12.xlsx') # dictionary path
INPUT_DATA_PATH = os.path.abspath('../../data/processed_input_data/02_modelling_stack_data_seasonality.csv') # modeling data stack
WRITER = pd.ExcelWriter('./Modeling_Insights_Deck_Input.xlsx', engine = 'xlsxwriter') # location for the Modeling_Insights_Deck_input report

# specify the model details
MODEL_DICT = {
    'sales_unit': 
                # input for modeling results
                {
                'output_folder': '../../models/manual/sales_unit/mix_media_metrics/2021-11-09_model_outputs_v1',
                'model_results_folder': '../../models/manual/sales_unit/mix_media_metrics/2021-11-09_model_outputs_v1/model_results_0802',     
                # input the start and end date for the s_curve reports
                'S_Curve_start_end_date': ('2019-01-01', '2021-03-31'),
                                              
                     # input for single optimizer (sheet 9 - SingleOpt, 10 - SingleOpt Summary)
                     'singleOpt_scenarios':
                    {
                             'singleOpt_1':
                            {
                                 'singleOpt_start_end_date': ('2020-02-01', '2021-01-31'),
                                 'singleOpt_df_new_file': './df_new_units.csv',
                                 'singleOpt_contrib_df_file': './contrib_df_units.csv', 
                                 'singleOpt_roas_def_file': './roas_def_units.csv',
                            },
                        # please provide the folder name of the optimiation outputs under this model result folder, e.g. '../2021-11-09_model_outputs_v1' here
                             'singleOpt_2':
                            {
                                 'singleOpt_start_end_date': ('2020-03-01', '2021-01-31'),
                                 'singleOpt_df_new_file': './df_new_units.csv',
                                 'singleOpt_contrib_df_file': './contrib_df_units.csv', 
                                 'singleOpt_roas_def_file': './roas_def_units.csv',
                            },
                        # please add/remove extra scenarios as needed

                    },
                 
                     # no need to change
                     'model_filter': 1,  
                     'model': 'OMNI Volume Sales Unit Model', # must follow the same naming convention, will be printed in Tableau
                     'subtitle': 'Sales Unit' # must follow the same naming convention, will be printed in Tableau
                },

    'new_customer': 
                # input for modeling results
                {
                'output_folder': '../../models/manual/new_customer/mix_media_metrics/2021-11-05_model_outputs_v2',
                'model_results_folder': '../../models/manual/new_customer/mix_media_metrics/2021-11-05_model_outputs_v2/model_results_customer_0802',                   
                # input the start and end date for the s_curve reports 
                'S_Curve_start_end_date': ('2019-01-01', '2021-03-31'),
                                              
                    # input for single optimizer (sheet 9 - SingleOpt, 10 - SingleOpt Summary)
                    'singleOpt_scenarios':
                    {
                            'singleOpt_1':
                            {
                                'singleOpt_start_end_date': ('2020-02-01', '2021-01-31'),
                                 'singleOpt_df_new_file': './df_new.csv',
                                 'singleOpt_contrib_df_file': './contrib_df.csv', 
                                 'singleOpt_roas_def_file': './roas_def.csv',
                            },
                            # please update start_end_date, df_new_file, contrib_df_file, roas_def_file for additional scenairos
                            'singleOpt_2':
                            {
                                'singleOpt_start_end_date': ('2020-03-01', '2021-01-31'),
                                 'singleOpt_df_new_file': './df_new.csv',
                                 'singleOpt_contrib_df_file': './contrib_df.csv', 
                                 'singleOpt_roas_def_file': './roas_def.csv',
                            },
                        # please add/remove extra scenarios as needed

                    },
                 
                    # no need to change 
                    'model_filter': 2, 
                    'model': 'OMNI New Customer Count Model', # must follow the same naming convention, will be printed in Tableau
                    'subtitle': 'New Customer' # must follow the same naming convention, will be printed in Tableau      
                } 
}

# input for joint Optimizer, read the sales_unit singleOpt start_end_time
JOINT_OPT_DICT = {
    'jointOpt_1': 
    {
        'jointOpt_start_end_date': ('2020-02-01', '2021-01-31'),
        'jointOpt_result_df_file': './result_df.csv', 
        'jointOpt_result_contrib_file': './result_contrib.csv'
    },
    # please update start_end_date, result_df_file, result_contrib_file for additional scenairos
    'jointOpt_2': 
    {
        'jointOpt_start_end_date': ('2020-03-01', '2021-01-31'), 
        'jointOpt_result_df_file': './result_df.csv', 
        'jointOpt_result_contrib_file': './result_contrib.csv'
    },
    # please add/remove extra scenarios as needed
        
}


# time period input for media contribution 
# (sheet 1 - WMC Portfolio Contribution, sheet 2 - WMC Performance, sheet 3 - WMC Media Contribution)
#'Period' value will be printed in Tableau
WMC_PORTFOLIO_DICT = {
     'WMC_entire': {'start_date':'2019-01-01', 'end_date':'2021-03-31', 'Period': '28 Months'},  
     'WMC_2019_Q4': {'start_date':'2019-10-01', 'end_date':'2019-12-31', 'Period': 'Q4 2019'}, 
     'WMC_2020_Q4': {'start_date':'2020-10-01', 'end_date':'2020-12-31', 'Period': 'Q4 2020'},
}

# time period input for growth analysis (sheet 6 - Growth Analysis)
DUE_TO_OUTCOME_DICT = {
    'due_to_outcome_2019_2020': {'start_date_1':'2019-01-01', 
                                 'end_date_1':'2019-12-31', 
                                 'start_date_2':'2020-01-01', 
                                 'end_date_2':'2020-12-31'},  
    'due_to_outcome_2020_2021': {'start_date_1':'2020-01-01', 
                                 'end_date_1':'2020-03-31', 
                                 'start_date_2':'2021-01-01', 
                                 'end_date_2':'2021-03-31'}
}

# initiate the stack dataset
data = pd.read_csv(INPUT_DATA_PATH, thousands= ",", na_values = " -   ")
data['index'] = pd.to_datetime(data['index']) 
data.set_index("index", inplace = True, drop= False) 
df = data.copy()

# Please specify your custom variables that were not part of standard data stack
# Example:

df['M_SEARCH_MAN_CLK']=df['M_SBA_CLK']+df['M_SP_KWB_CLK']
df['M_SEARCH_AUTO_CLK']=df['M_SP_AB_CLK']
df['M_NATIONAL_TV_SPEND']=1000 
df['M_SEARCH_MAN_SPEND']=df['M_SBA_SPEND']+df['M_SP_KWB_SPEND']
df['M_SEARCH_AUTO_SPEND']=df['M_SP_AB_SPEND']
df['RATE_ON_DIS_CT_ROS']=df['M_ON_DIS_CT_ROS_SPEND']/df['M_ON_DIS_CT_ROS_IMP']
df['RATE_SEARCH_AUTO_CLK']=df['M_SEARCH_AUTO_SPEND']/df['M_SEARCH_AUTO_CLK']
df['RATE_SEARCH_MAN_CLK']=df['M_SEARCH_MAN_SPEND']/df['M_SEARCH_MAN_CLK']
df['RATE_NATIONAL_TV']=df['M_NATIONAL_TV_SPEND']/df['M_NATIONAL_TV_IMP']


## Helper functions

In [5]:
# create values for the column 'Time Range' from the report, e.g. Jan'19 - Jan'21
def get_time_range(start_date_str, end_date_str):
        time_range_ls = []
        for date_str in [start_date_str, end_date_str]:
            date_time_obj = datetime.strptime(date_str, '%Y-%m-%d')
            month = date_time_obj.strftime("%b")
            year = date_time_obj.strftime("%y")
            res = "'".join([month, year])
            time_range_ls.append(res)
        time_range = " - ".join(time_range_ls)
        return time_range
    
def get_time_range_comparison(time_range_1, time_range_2):
    time_range_comparison = ' vs '.join([time_range_1, time_range_2])
    return time_range_comparison

# create values for the column 'Period' from the report, e.g. 30 Months
def get_period_value(start_date_str, end_date_str):
        start_date = datetime.strptime(start_date_str, '%Y-%m-%d')
        end_date = datetime.strptime(end_date_str, '%Y-%m-%d')
        num_months = (end_date.year - start_date.year) * 12 + (end_date.month - start_date.month) + 1
        return '{} Months'.format(num_months)
    
def get_due_to_outcome_attribute(period): 
    components = period.split('_')
    if components[-3].startswith('Q'):
        idx_val_1 = ' '.join([components[-3], components[-2]])
        idx_val_2 = ' '.join([components[-3], components[-1]])
    else:
        idx_val_1 = components[-2]
        idx_val_2 = components[-1]    
    return idx_val_1,idx_val_2

def get_var_description_mapping():    
        wamm_mapping = pd.read_excel(WAMM_DICT_PATH, sheet_name = 'Data Dictionary')
        return {n:d for n,d in zip(wamm_mapping['Variable Name'].values,wamm_mapping['Variable Description'].values)}

VAR_DESCRIPTION_MAPPING = get_var_description_mapping()

def get_var_description(var_name):
    if var_name in VAR_DESCRIPTION_MAPPING:
        return VAR_DESCRIPTION_MAPPING[var_name]
    elif var_name == 'Total':
        return
    else:
        return var_name

def combine_columns(row, col_A, col_B):
    if col_A in row and col_B not in row:
        return row[col_A]
    elif col_A not in row and col_B in row:
        return row[col_B]
    elif col_A in row and col_B in row:
        if pd.isna(row[col_A]):
            return row[col_B]
        else:
            return row[col_A]
    else:
        return 
    
def is_float(df, column):
    try:
        df[column] = df[column].apply(lambda x : float(x))
    except ValueError:
        return False
    return True

def is_int(val):
    try:
        num = int(val)
    except ValueError:
        return False
    return True

Error_msg_ls = []

## Modeling_Insights_Deck_Input file processing 

### Sheet 1:  WMC Portfolio Contribution
The purpose of this block is to generate the data for sheet: WMC Portfolio Contribution

In [6]:
# Here I will use the WMC_PORTFOLIO_DICT that entered at the beginning

In [7]:
# get per period/scenario WMC dataframe
def get_per_scenario_WMC_dataframe(model, scenario, period_filter):
    
    base_path = MODEL_DICT[model]['output_folder']
    start_date_str = WMC_PORTFOLIO_DICT[scenario]['start_date']
    end_date_str = WMC_PORTFOLIO_DICT[scenario]['end_date']
    sub_path = 'WMC_' + str(start_date_str) + '_'+ str(end_date_str) + '.csv'
    file_path = os.path.join(base_path, sub_path)            
    try:
        WMC_df = pd.read_csv(file_path)
        WMC_df['Model Filter'] = MODEL_DICT[model]['model_filter']
        WMC_df['Model'] = MODEL_DICT[model]['model']
        WMC_df['Subtitle'] = MODEL_DICT[model]['subtitle']
        WMC_df['Period Filter'] = period_filter
        WMC_df['Time Range'] = get_time_range(start_date_str, end_date_str)
        WMC_df['Period'] = WMC_PORTFOLIO_DICT[scenario]['Period']
        return WMC_df
    except:
        msg = 'Warning! {model} model - {scenario}: {sub_path} does not exist! Please double check the start and end dates!\n'.format(model = model, scenario=scenario, sub_path = sub_path)
        Error_msg_ls.append(msg)
        print(msg)
        return

In [8]:
# update get per period/scenario WMC contribution dataframe
def update_per_scenario_WMC_contribution_dataframe(WMC_contribution_df):
    if WMC_contribution_df is None:
        return 
    # only keep the rows of Base and Incremental 
    WMC_contribution_df = WMC_contribution_df[WMC_contribution_df['Display Level'].isin(['Total Incremental', 'Base'])]
    WMC_contribution_df['Display Level'] = WMC_contribution_df['Display Level'].replace({'Total Incremental': 'Incremental'}) 
    WMC_contribution_df.rename(columns = {'Contribution Share (%)': 'Share %', 
                              'Display Level': 'Base/Incremental', 
                              'Total Sales ($)': 'Sales', 
                              'Total Contribution': 'Volume Units'}, inplace = True)    
    # convert the string columns into float
    WMC_contribution_df['Volume Units'] = WMC_contribution_df['Volume Units'].apply(lambda x : round(float(str(x).replace(',', '')), 2))
    WMC_contribution_df['Sales'] = WMC_contribution_df['Sales'].apply(lambda x : round(float(str(x).replace(',', '')), 2))
    WMC_contribution_df = WMC_contribution_df.sort_values(by = ['Period Filter', 'Base/Incremental'], ascending = [True, True])
    
    # reorder the DataFrame based on the columns
    cols = ['Model Filter', 'Model', 'Subtitle', 'Period Filter', 'Period', 'Time Range', 'Base/Incremental', 'Share %', 'Volume Units', 'Sales']
    return WMC_contribution_df[cols]

In [9]:
# get all_scenario_WMC contribution dataframes from one model
def get_per_model_WMC_contribution_dataframe(model):
    per_model_WMC_contribution_ls = []
    period_filter = 1
    for scenario in WMC_PORTFOLIO_DICT:
        WMC_contribution_df = get_per_scenario_WMC_dataframe(model, scenario, period_filter)
        if WMC_contribution_df is not None:
            updated_WMC_contribution_df = update_per_scenario_WMC_contribution_dataframe(WMC_contribution_df)
            per_model_WMC_contribution_ls.append(updated_WMC_contribution_df)
            period_filter += 1

    if per_model_WMC_contribution_ls:
        per_model_WMC_contribution_df = pd.concat(per_model_WMC_contribution_ls)
        return per_model_WMC_contribution_df

In [10]:
# combine dataframes from all models
def combine_WMC_contribution_dataframes():

    WMC_contribution_ls = []
    for model in MODEL_DICT:
        per_model_WMC_contribution = get_per_model_WMC_contribution_dataframe(model)
        WMC_contribution_ls.append(per_model_WMC_contribution)
    if  WMC_contribution_ls:
        WMC_contribution_df = pd.concat(WMC_contribution_ls)
        return WMC_contribution_df

In [11]:
# combine the dataframes from all models
combined_WMC_contribution_df =  combine_WMC_contribution_dataframes()

# save the data with the below sheetname
combined_WMC_contribution_df.to_excel(WRITER, sheet_name = 'WMC Portfolio Contribution', index = False)

### Sheet 2: WMC Performance
The purpose of this block is to generate the data for sheet: WMC Performance

In [12]:
# Here I will use the WMC_PORTFOLIO_DICT that entered at the beginning

In [13]:
# update get per period/scenario WMC performance dataframe
def update_per_scenario_WMC_performance_dataframe(WMC_performance_df):
    if WMC_performance_df is None:
        return 
    # only keep the rows for Total Incremental 
    WMC_performance_df = WMC_performance_df[WMC_performance_df['Display Level'] == 'Total Incremental']
    
    # only keep the required columns
    WMC_performance_df.rename(columns = {'Total Sales ($)': 'WMT Sales', 
                                         'Total Spend ($)': 'WMC Spend'}, inplace = True)
    # convert the string into decimals for computation
    WMC_performance_df['WMT Sales'] = WMC_performance_df['WMT Sales'].apply(lambda x : float(x.replace(',', '')))
    WMC_performance_df['WMC Spend'] = WMC_performance_df['WMC Spend'].apply(lambda x : float(x.replace(',', '')))
    WMC_performance_df['Total Contribution'] = WMC_performance_df['Total Contribution'].apply(lambda x : float(x.replace(',', '')))
    
    # create a new column iROAS = WMC Sales/WMC Spend, and round the column
    WMC_performance_df['iRoAS'] = WMC_performance_df['WMT Sales']/WMC_performance_df['WMC Spend']
    WMC_performance_df['iRoAS'] = WMC_performance_df['iRoAS'].apply(lambda x: round(x, 2))
    
    # create a new columns CPIA = Total Spend/Total Contribution, and round the column
    WMC_performance_df['CPIA'] = WMC_performance_df['WMC Spend']/WMC_performance_df['Total Contribution']
    WMC_performance_df['CPIA'] = WMC_performance_df['CPIA'].apply(lambda x: round(x, 2))
    
    # drop the unnecessary columns
    WMC_performance_df.drop(columns = ['Display Level'], inplace = True)
    
    # reorder the DataFrame based on the columns
    cols = ['Model Filter', 'Period', 'Period Filter', 'Time Range', 'Model', 'WMT Sales', 'WMC Spend',  'iRoAS', 'CPIA']
    return WMC_performance_df[cols]

In [14]:
# get all_scenario_WMC performance dataframes from one model
def get_per_model_WMC_performance_dataframe(model):
    per_model_WMC_performance_ls = []
    period_filter = 1
    for scenario in WMC_PORTFOLIO_DICT:
        WMC_performance_df = get_per_scenario_WMC_dataframe(model, scenario, period_filter)
        if WMC_performance_df is not None:
            updated_WMC_performance_df = update_per_scenario_WMC_performance_dataframe(WMC_performance_df)
            per_model_WMC_performance_ls.append(updated_WMC_performance_df)
            period_filter += 1

    if per_model_WMC_performance_ls:
        per_model_WMC_performance_df = pd.concat(per_model_WMC_performance_ls)
        return per_model_WMC_performance_df

In [15]:
# combine dataframes from all models
def combine_WMC_performance_dataframes():

    WMC_performance_ls = []
    for model in MODEL_DICT:
        per_model_WMC_performance = get_per_model_WMC_performance_dataframe(model)
        WMC_performance_ls.append(per_model_WMC_performance)
    if  WMC_performance_ls:
        WMC_performance_df = pd.concat(WMC_performance_ls)
        return WMC_performance_df

In [16]:
# combine the dataframes from all models
combined_WMC_performance_df =  combine_WMC_performance_dataframes()

# save the data to the below sheetname
combined_WMC_performance_df.to_excel(WRITER, sheet_name = 'WMC Performance', index = False)

### Sheet 3: WMC Media Contribution
The purpose of this block is to generate the data for sheet: WMC Media Contribution

In [17]:
# Here I will use the WMC_PORTFOLIO_DICT that entered at the beginning

In [18]:
# update get per period/scenario WMC media contribution dataframe
def update_per_scenario_WMC_media_contribution_dataframe(WMC_media_contribution_df):
    if WMC_media_contribution_df is None:
        return 
    WMC_media_contribution_df['Value RoAS ($)'] = WMC_media_contribution_df.apply (lambda row: combine_columns(row, 'Value NCoAS ($)', 'Value RoAS ($)'), axis=1)
    WMC_media_contribution_df['Volume RoAS'] = WMC_media_contribution_df.apply (lambda row: combine_columns(row, 'NCoAS', 'Volume RoAS'), axis=1)
    WMC_media_contribution_df = WMC_media_contribution_df[~WMC_media_contribution_df['Display Level'].isin(['Total Incremental', 'Base', 'Total'])]
        
    # convert the string columns into float
    str_cols = ['Total Contribution', 'Total Sales ($)', 'Total Spend ($)', 'Raw Media']
    #check if the column could be converted to float, if yes, convert to float
    for str_col in str_cols:   
        WMC_media_contribution_df[str_col] = WMC_media_contribution_df[str_col].apply(lambda x : x.replace(',', ''))
        if is_float(WMC_media_contribution_df, str_col):
            WMC_media_contribution_df[str_col] = WMC_media_contribution_df[str_col].apply(lambda x : float(x))
    WMC_media_contribution_df.replace([np.inf, -np.inf], np.nan, inplace=True)    
    # reorder the DataFrame based on the columns
    cols = ['Model Filter', 'Model', 'Period Filter', 'Period', 'Time Range', 'Display Level',
            'Contribution Share (%)', 'Total Contribution', 'Total Sales ($)', 'Total Spend ($)', 
           'Spend Share (%)', 'Raw Media', 'Raw Media Share (%)', 'Volume RoAS', 'Value RoAS ($)']
    return WMC_media_contribution_df[cols]

In [19]:
# get all_scenario_WMC_media_contribution dataframes from one model
def get_per_model_WMC_media_contribution_dataframe(model):
    per_model_WMC_media_contribution_ls = []
    period_filter = 1
    for scenario in WMC_PORTFOLIO_DICT:
        WMC_media_contribution_df = get_per_scenario_WMC_dataframe(model, scenario, period_filter)
        if WMC_media_contribution_df is not None:
            updated_WMC_media_contribution_df = update_per_scenario_WMC_media_contribution_dataframe(WMC_media_contribution_df)
            per_model_WMC_media_contribution_ls.append(updated_WMC_media_contribution_df)
            period_filter += 1
    if per_model_WMC_media_contribution_ls:
        per_model_WMC_media_contribution_df = pd.concat(per_model_WMC_media_contribution_ls)
        return per_model_WMC_media_contribution_df

In [20]:
# combine dataframes from all models
def combine_WMC_media_contribution_dataframes():

    WMC_media_contribution_ls = []
    for model in MODEL_DICT:
        per_model_WMC_media_contribution = get_per_model_WMC_media_contribution_dataframe(model)
        WMC_media_contribution_ls.append(per_model_WMC_media_contribution)
    if  WMC_media_contribution_ls:
        WMC_media_contribution_df = pd.concat(WMC_media_contribution_ls)
        return WMC_media_contribution_df

In [21]:
# update the WMC media contribution dataset
combined_WMC_media_contribution_df = combine_WMC_media_contribution_dataframes()

# Save the data with the below sheetname: 
combined_WMC_media_contribution_df.to_excel(WRITER, sheet_name = 'WMC Media Contribution', index = False)

### Sheet 4: Model Act vs Pred Modeling
The purpose of this block is to generate the data for sheet: Model Act vs Pred Modeling

In [22]:
# get the act_pred_comparison dataframe for one model
def get_act_pred_comparison_dataframe(model):
    # read the model_output
    base_path = MODEL_DICT[model]['model_results_folder']
    file_path = os.path.join(base_path, 'model_outputs.xlsx')
    modelling_df = pd.read_excel(file_path, sheet_name = 'Act vs Pred Modelling')
    validation_df = pd.read_excel(file_path, sheet_name = 'Act vs Pred Validation')
    # rename the columns for modeling and validation dataframe
    modelling_df.rename(columns = {'Actuals': 'Modeling Actual', 'Predicted':'Modeling Predicted'}, inplace = True) 
    validation_df.rename(columns = {'Actuals': 'Validation Actual', 'Predicted':'Validation Predicted'}, inplace = True)
    # combine the modeling and validation dataframes
    modelling_comparison_df = modelling_df.append(validation_df, ignore_index=True)
    
    # add the filter columns
    modelling_comparison_df['Model Filter'] = MODEL_DICT[model]['model_filter']
    modelling_comparison_df['Model'] = MODEL_DICT[model]['model']
    # convert the datetime from str into datetime obj
    modelling_comparison_df['index'] = modelling_comparison_df['index'].apply(lambda x : datetime.strptime(x, '%Y-%m-%d'))

    # reorder the DataFrame based on the columns
    cols = ['Model Filter', 'Model', 'index', 'Modeling Actual', 'Modeling Predicted', 'Residual', 'Validation Actual', 'Validation Predicted']
    return modelling_comparison_df[cols]

In [23]:
# combine the dataframes from all models
def combine_modelling_comparison_dataframes():
    modelling_comparison_ls = []
    for model in MODEL_DICT:
        modelling_comparison = get_act_pred_comparison_dataframe(model)
        modelling_comparison_ls.append(modelling_comparison)
    
    modelling_comparison_df = pd.concat(modelling_comparison_ls)
    return modelling_comparison_df

In [24]:
# combine the dataframes from all models
combined_modelling_comparison_df = combine_modelling_comparison_dataframes()

# Save the data with the below sheetname: 
combined_modelling_comparison_df.to_excel(WRITER, sheet_name = 'Model Act vs Pred Modelling', index = False)

### Sheet 5. Model MAPE & R Square
The purpose of this block is to generate the data for sheet: Model MAPE & R Square

In [25]:
def get_modelling_metrics_dataframe(model):
    # read the model_output
    base_path = MODEL_DICT[model]['model_results_folder']
    file_path = os.path.join(base_path, 'model_outputs.xlsx')
    modelling_metrics = pd.read_excel(file_path, sheet_name = 'Model Stats')
    modelling_metrics['Model Filter'] = MODEL_DICT[model]['model_filter']
    modelling_metrics['Model'] = MODEL_DICT[model]['model']
    modelling_metrics = modelling_metrics.reset_index(drop = True)
    # reorder the columns
    cols = ['Model Filter', 'Model', 'Metric', 'Model Fit', 'Validation Fit']
    return modelling_metrics[cols]

In [26]:
def combine_modelling_metrics_dataframes():
    modelling_metrics_ls = []
    for model in MODEL_DICT:
        modelling_metrics = get_modelling_metrics_dataframe(model)
        modelling_metrics_ls.append(modelling_metrics)
        
    modelling_metrics_df = pd.concat(modelling_metrics_ls)
    return modelling_metrics_df

In [27]:
# conbine the dataframes from all models
combined_modelling_metrics_df = combine_modelling_metrics_dataframes()

# Save the data with the below sheetname: 
combined_modelling_metrics_df.to_excel(WRITER, sheet_name = 'Model MAPE & R Square', index = False)

### Sheet 6. Growth Analysis
The purpose of this block is to generate the data for sheet: Growth Analysis

In [28]:
# Here I will use the DUE_TO_OUTCOME_DICT that entered at the beginning

In [29]:
def get_due_to_outcome_attributes(period): 
    idx_val_1 = 'Period 1' 
    idx_val_2 = 'Period 2'
    comps = sorted(list(set([com for com in period.rsplit('_')][3:])))
    if len(comps) < 2 or len(comps) > 3:
        return idx_val_1, idx_val_2
    
    elif len(comps) == 2:
        idx_val_1 = comps[0]
        idx_val_2 = comps[1]
    else:
        comp_1, comp_2, comp_3 = comps[0], comps[1], comps[2]
        if is_int(comp_1):
            if comp_1 < comp_2:
                idx_val_1 = ' '.join([comp_1, comp_3])
                idx_val_2 = ' '.join([comp_2, comp_3])
            else:
                idx_val_2 = ' '.join([comp_1, comp_3])
                idx_val_1 = ' '.join([comp_2, comp_3])     
        else:
            if comp_1 < comp_2:
                idx_val_1 = ' '.join([comp_2, comp_1])
                idx_val_2 = ' '.join([comp_3, comp_1])
            else:
                idx_val_2 = ' '.join([comp_2, comp_1])
                idx_val_1 = ' '.join([comp_3, comp_1])
    return idx_val_1, idx_val_2

In [30]:
# get per period/scenario due_to_outcome dataframe
def get_per_scenario_due_to_outcome_dataframe(model, scenario):
    
    base_path = MODEL_DICT[model]['output_folder']
    start_date_str_1 = DUE_TO_OUTCOME_DICT[scenario]['start_date_1']
    end_date_str_1 = DUE_TO_OUTCOME_DICT[scenario]['end_date_1']
    start_date_str_2 = DUE_TO_OUTCOME_DICT[scenario]['start_date_2']
    end_date_str_2 = DUE_TO_OUTCOME_DICT[scenario]['end_date_2']
    sub_path = 'WMC_Due_to_' + str(start_date_str_1) + '_' + str(end_date_str_1) + '_' + str(start_date_str_2) + '_'+ str(end_date_str_2)+ '.csv'
    path = os.path.join(base_path, sub_path)
    try:
        df = pd.read_csv(path)

        rows = ['Number of stores selling brand/Number of Walmart stores in U.S.*100', 
                'Out of stock store count over replenishable store count ratio', 
                'Outcome Variable', 
               'Days/Holidays/Events', 
               'Seasonality']
        due_to_outcome_df = df[~df['Unnamed: 0'].isin(rows)]
        # convert the str_cols into float
        str_cols = ['Due To', 'Due To (%)', 'Spend Period 1', 'Spend Period 2']
        for str_col in str_cols:
            due_to_outcome_df[str_col] = due_to_outcome_df[str_col].apply(lambda x : float(str(x).replace(',', '')))

        due_to_outcome_df.sort_values(by = 'Due To', ascending = True, inplace = True)
        due_to_outcome_df.reset_index(drop = True, inplace = True)

        # the comparison values 
        p1 = df[df['Unnamed: 0'] == 'Outcome Variable']['Contribution Period 1'].values[0]
        p2 = df[df['Unnamed: 0'] == 'Outcome Variable']['Contribution Period 2'].values[0]
        # add Attributes values, e.g. 2020 Q4, 2019 Q4
        idx_val_1, idx_val_2 = get_due_to_outcome_attributes(scenario)
        df1 = pd.DataFrame({'Unnamed: 0': [idx_val_1], 'Due To': [p1]})
        df2 = pd.DataFrame({'Unnamed: 0': [idx_val_2], 'Due To': [p2]})
        due_to_outcome_df = df1.append(due_to_outcome_df, ignore_index = True).fillna(np.nan)
        due_to_outcome_df = due_to_outcome_df.append(df2, ignore_index = True).fillna(np.nan)
        due_to_outcome_df[-1:]['Spend Period 1'] = due_to_outcome_df[1:-1]['Spend Period 1'].sum()
        due_to_outcome_df[-1:]['Spend Period 2'] = due_to_outcome_df[1:-1]['Spend Period 2'].sum()

        # create the time_range_comparison
        time_range_1 = get_time_range(start_date_str_1, end_date_str_1) 
        time_range_2 =get_time_range(start_date_str_2, end_date_str_2)
        due_to_outcome_df['Comparison'] = get_time_range_comparison(time_range_1, time_range_2)
        return due_to_outcome_df
    except:
        msg = 'Warning! {model} model - {scenario}: {sub_path} does not exist! Please double check the start and end dates!\n'.format(model = model, scenario=scenario, sub_path = sub_path)
        Error_msg_ls.append(msg)
        print(msg)
        return

In [31]:
# update the due_to_outcome_dataframe by adding the filter columns, rename the column names
def update_per_scenario_due_to_outcome_dataframe(model, due_to_outcome_df, period_filter):
    if due_to_outcome_df is None:
        return 
    due_to_outcome_df.rename(columns = {'Due To': 'Cashflow', 
                                        'Due To (%)': '% change',
                                       'Unnamed: 0': 'Attributes'}, inplace = True)
    due_to_outcome_df['Cashflow'] = due_to_outcome_df['Cashflow'].apply(lambda x : float(str(x).replace(',', '')))
    due_to_outcome_df['Attributes'] = due_to_outcome_df['Attributes'].apply(lambda x : x.strip())
    due_to_outcome_df.dropna(subset = ['Attributes'], inplace=True)

    # add filter columns
    due_to_outcome_df['Model Filter'] = MODEL_DICT[model]['model_filter']
    due_to_outcome_df['Model'] = MODEL_DICT[model]['model']
    due_to_outcome_df['Period Filter'] = period_filter
    due_to_outcome_df['sort'] = list(range(1, len(due_to_outcome_df) + 1))
    # reorder the columns
    cols = ['Model Filter', 'Period Filter', 'Model', 'Comparison', 'Attributes', 'Cashflow',  'sort', '% change', 'Spend Period 1', 'Spend Period 2']
    return due_to_outcome_df[cols]

In [32]:
# get all_scenario_due_to_outcome dataframes from one model
def get_per_model_due_to_outcome_dataframe(model):
    per_model_due_to_outcome_ls = []
    period_filter = 1
    for scenario in DUE_TO_OUTCOME_DICT:
        due_to_outcome = get_per_scenario_due_to_outcome_dataframe(model, scenario)
        if due_to_outcome is not None:
            updated_due_to_outcome = update_per_scenario_due_to_outcome_dataframe(model, due_to_outcome, period_filter)
            per_model_due_to_outcome_ls.append(updated_due_to_outcome)
            period_filter += 1

    if per_model_due_to_outcome_ls:
        per_model_due_to_outcome_df = pd.concat(per_model_due_to_outcome_ls)
        return per_model_due_to_outcome_df

In [33]:
# combine dataframes from all models
def combine_due_to_outcome_dataframes():

    due_to_outcome_ls = []
    for model in MODEL_DICT:
        per_model_due_to_outcome = get_per_model_due_to_outcome_dataframe(model)
        due_to_outcome_ls.append(per_model_due_to_outcome)
    if  due_to_outcome_ls:
        due_to_outcome_df = pd.concat(due_to_outcome_ls)
        return due_to_outcome_df

In [34]:
# combine all datasets
combined_due_to_outcome_df = combine_due_to_outcome_dataframes()

# Save the data with the below sheetname: 
combined_due_to_outcome_df.to_excel(WRITER, sheet_name = 'Growth Analysis', index = False)

### Sheet 7. Stack
The purpose of this block is to generate the data for sheet: Stack

In [35]:
#cols = df.columns.tolist()[:-18]
df_stack = df.copy()
df_stack.reset_index(drop=True, inplace=True)

# Save the data with the below sheetname: 
df_stack.to_excel(WRITER, sheet_name = 'Stack', index = False)

### Sheet 8 - 1. Pacing
The purpose of this block is to generate the data for sheet: Pacing

In [36]:
# get per model pacing dataframe
def get_pacing_dataframe(model):
    base_path = MODEL_DICT[model]['model_results_folder']
    file_path = os.path.join(base_path, 'variables_transformation_summary.xlsx')
    points_df = pd.read_excel(file_path, sheet_name = 'Points')    
    spend_points_df = pd.read_excel(file_path, sheet_name = 'Spends Points')    
    # update the columns name of the points_df and spend_points_df
    points_df['Variable'] = points_df['Variable'].apply(lambda x: x.rsplit('_', 5)[0])
    spend_points_df['Variable'] = spend_points_df['Variable'].apply(lambda x: x.rsplit('_', 5)[0].replace('_VIEW_IMP', '_SPEND').replace('_IMP', '_SPEND').replace('_CLK', '_SPEND'))
    
    # read the stack dataset
    pacing_df = df.copy()
    pacing_point_df = pacing_df[['index'] + list(points_df['Variable'].values)]
    pacing_spend_df = pacing_df[['index'] + list(spend_points_df['Variable'].values)]
    
    return pacing_point_df, pacing_spend_df, points_df, spend_points_df

In [37]:
# pivot the pacing dataframe
def pivot_per_model_pacing_dataframe(dataset):
    
    metric_dict = {'IMP': 'Impressions',
                   'CLK': 'Clicks',
                   'SPEND': 'Spend'}

    def get_metric(x):
        y = x.rsplit('_', 1)[-1]
        if y in metric_dict:
            return metric_dict[y]
        else:
            return 'Other'
    
    onAirInput_ls = []
    for metric in dataset.columns:
        if metric == 'index':
            continue
        per_metric_df = dataset.melt(id_vars = ['index'], value_vars = [metric], ignore_index = False)
        onAirInput_ls.append(per_metric_df)
        
    if onAirInput_ls:
        onAirInput_df = pd.concat(onAirInput_ls, ignore_index = True)
        onAirInput_df['Metric'] = onAirInput_df['variable'].apply(get_metric)
        onAirInput_df.rename(columns = {'variable': 'Media Driver', 'value': 'Value'}, inplace = True)
        return onAirInput_df

In [38]:
# update the Variable values
def update_per_model_pacing_dataframe(model, pacing_df, points_df):

    # Add Saturation, Inflection, Threshold from report, variables_transformation_summary
    result = pacing_df.merge(points_df[['Variable','Saturation Point', 'Threshold Point', 'Inflection Point']], 
                          how = 'left', left_on = 'Media Driver', right_on = 'Variable' )
    #result.drop(columns = ['Variable'], inplace = True)
    result = result.fillna(0)
    # add model filter
    result['Model Filter'] = MODEL_DICT[model]['model_filter']
    result['Model'] = MODEL_DICT[model]['model']
    result['WAMM Tactic'] = result['Media Driver'].apply(get_var_description)
    cols = ['Model Filter', 'Model', 'index',  'Media Driver', 'WAMM Tactic','Metric', 'Value','Saturation Point', 'Threshold Point', 'Inflection Point']
    result = result[cols]
    return result

In [39]:
# get the result dataframe for one model
def get_per_model_dataframe(model):
    
    pacing_point_df, pacing_spend_df, points_df, spend_points_df = get_pacing_dataframe(model)
    # pivot the point and spend values for each model
    pacing_point_pivot_df = pivot_per_model_pacing_dataframe(pacing_point_df)
    pacing_spend_pivot_df = pivot_per_model_pacing_dataframe(pacing_spend_df)

    # update the point and spend values for each model
    result_point = update_per_model_pacing_dataframe(model, pacing_point_pivot_df, points_df)
    result_spend = update_per_model_pacing_dataframe(model, pacing_spend_pivot_df, spend_points_df)
    
    # combine the point and spend values for each model
    per_model_dataset = pd.concat([result_point, result_spend])
    return per_model_dataset

In [40]:
# combine dataframes from all models
def combine_pacing_dataframes():
    pacing_ls = []
    for model in MODEL_DICT:
        pacing = get_per_model_dataframe(model)
        pacing_ls.append(pacing)
    pacing_df = pd.concat(pacing_ls)
    return pacing_df

In [41]:
# combine the dataframes from all models
combined_pacing_dataframes = combine_pacing_dataframes()

# Save the data with the below sheetname: 
combined_pacing_dataframes.to_excel(WRITER, sheet_name = 'Pacing', index = False)

### Sheet 8 - 2. onAir
The purpose of this block is to generate the data for sheet: onAir

In [42]:
def get_onAir_dataframe(dataset):
    sub_dataset = dataset[[col for col in dataset.columns if col == 'index' or col.endswith('_IMP')]]
    filter_cols = sub_dataset.columns.str.contains('_OG_|_DOTCOM_|_VIEW_IMP')
    filtered_dataset = sub_dataset[sub_dataset.columns[~filter_cols]]
    return filtered_dataset

In [43]:
# update the Variable values
def update_onAir_dataframe(dataset):
    
    result = dataset.copy()
    result.rename(columns = {'Value': 'Impressions', 'Media Driver' : 'Source'}, inplace = True)
    result = result.fillna(0)   
    result['WAMM Tactic'] = result['Source'].apply(get_var_description)
    cols = ['index',  'Impressions', 'Source', 'WAMM Tactic']
    return result[cols]

In [44]:
# read data from the sheet 7: df_stack
dataset = df_stack.copy()
onAir_df =  get_onAir_dataframe(dataset)
onAir_df = pivot_per_model_pacing_dataframe(onAir_df)
onAir_dataframe = update_onAir_dataframe(onAir_df)

# Save the data with the below sheetname: 
onAir_dataframe.to_excel(WRITER, sheet_name = 'onAir', index = False)

### Sheet 9. SingleOpt
The purpose of this block is to generate the data for sheet: SingleOpt

In [45]:
# get per scenario singleOpt dataframe
def get_per_scenario_singleOpt_dataframe(model, scenario):
    file_path = os.path.abspath(MODEL_DICT[model]['singleOpt_scenarios'][scenario]['singleOpt_df_new_file'])
    df_new = pd.read_csv(file_path)
    df_new.drop(df_new.columns[[0]], axis = 1, inplace = True)
    return df_new

In [46]:
# update per scenario singleOpt dataframe
def update_per_scenario_singleOpt_dataframe(model,scenario, df_new, period_filter):
    start_date_str, end_date_str = MODEL_DICT[model]['singleOpt_scenarios'][scenario]['singleOpt_start_end_date']
    df_new['Model Filter'] = MODEL_DICT[model]['model_filter']
    df_new['Model'] = MODEL_DICT[model]['model']
    df_new['Period Filter'] = period_filter
    df_new['Period'] = get_period_value(start_date_str, end_date_str)
    df_new['Time Range'] = get_time_range(start_date_str, end_date_str) 
    df_new['WAMM Tactic'] = df_new['Variable'].apply(get_var_description)
    # convert the string columns into float
    str_cols = ['Og Spends', 'Og Contri.', 'Og Rev', 'Opt Spends', 'Opt Contri.', 'Opt Rev']
    for col in str_cols:
        df_new[col] = df_new[col].apply(lambda x : round(float(str(x).replace(',', '')), 2))
    cols = ['Model Filter', 'Model', 'Period Filter', 'Period', 'Time Range', 'WAMM Tactic'] + df_new.columns.to_list()[:-6]
    df_new = df_new[cols]
    return df_new

In [47]:
# combine the singleOpt dataframes from all scenarios for one model
def get_per_model_singleOpt_dataframe(model):
    per_model_result_ls = []
    period_filter = 1
    for scenario in MODEL_DICT[model]['singleOpt_scenarios']:
        df_new = get_per_scenario_singleOpt_dataframe(model, scenario)
        updated_df_new = update_per_scenario_singleOpt_dataframe(model,scenario, df_new, period_filter)
        per_model_result_ls.append(updated_df_new)
        period_filter += 1
    
    if per_model_result_ls:
        per_model_result_df = pd.concat(per_model_result_ls)
        return per_model_result_df     

In [48]:
# combine_singleOpt_dataframes from all models
def combine_singleOpt_dataframes():
    singleOpt_ls = []
    for model in MODEL_DICT:
        df_new = get_per_model_singleOpt_dataframe(model)
        singleOpt_ls.append(df_new)

    singleOpt_df = pd.concat(singleOpt_ls)
    return singleOpt_df 

In [49]:
# combine the dataframes from all models
combined_single_opt_df=combine_singleOpt_dataframes()

# Save the data with the below sheetname: 
combined_single_opt_df.to_excel(WRITER, sheet_name = 'SingleOpt', index = False)

### Sheet 10. SingleOpt_Summary
The purpose of this block is to generate the data for sheet: SingleOpt_Summary

In [50]:
# get per scenario singleOpt_summary dataframe
def get_per_scenario_singleOpt_summary_dataframe(model, scenario):
    
    contrib_df_path = os.path.abspath(MODEL_DICT[model]['singleOpt_scenarios'][scenario]['singleOpt_contrib_df_file'])
    roas_df_path = os.path.abspath(MODEL_DICT[model]['singleOpt_scenarios'][scenario]['singleOpt_roas_def_file'])
    # load and clear contrib_df
    contrib_df = pd.read_csv(contrib_df_path)
    contrib_df.drop(contrib_df.columns[[0]], axis = 1, inplace = True)
    # load and clear roas_df
    roas_df = pd.read_csv(roas_df_path)
    roas_df.drop(roas_df.columns[[0]], axis = 1, inplace = True)
    return contrib_df, roas_df

In [51]:
# update per scenario singleOpt_summary dataframe
def update_per_scenario_singleOpt_summary_dataframe(model, scenario, contrib_df, roas_df, period_filter):
    result = pd.concat([contrib_df, roas_df], axis=1)
    start_date_str, end_date_str = MODEL_DICT[model]['singleOpt_scenarios'][scenario]['singleOpt_start_end_date']
    result['Model Filter'] = MODEL_DICT[model]['model_filter']
    result['Model'] = MODEL_DICT[model]['model']
    result['Period Filter'] = period_filter
    result['Period'] = get_period_value(start_date_str, end_date_str)
    result['Time Range'] = get_time_range(start_date_str, end_date_str) 
    result['%age change'] = result['%age change'].apply(lambda x: round(x, 2))
    cols = ['Model Filter', 'Model', 'Period Filter', 'Period', 'Time Range'] + result.columns.to_list()[:-5]
    result = result[cols]
    return result

In [52]:
# combine the singleOpt_summary dataframes from all scenarios for one model
def get_per_model_singleOpt_summary_dataframe(model):
    per_model_result_ls = []
    period_filter = 1
    for scenario in MODEL_DICT[model]['singleOpt_scenarios']:
        contrib_df, roas_df = get_per_scenario_singleOpt_summary_dataframe(model, scenario)
        updated_result = update_per_scenario_singleOpt_summary_dataframe(model, scenario, contrib_df, roas_df, period_filter)
        per_model_result_ls.append(updated_result)
        period_filter += 1
    if per_model_result_ls:
        per_model_result_df = pd.concat(per_model_result_ls)
        return per_model_result_df     

In [53]:
# combine_singleOpt_dataframes from all models
def combine_singleOpt_summary_dataframes():
    singleOpt_summary_ls = []
    for model in MODEL_DICT:
        singleOpt_summary = get_per_model_singleOpt_summary_dataframe(model)
        singleOpt_summary_ls.append(singleOpt_summary)
    singleOpt_summary_df = pd.concat(singleOpt_summary_ls)
    return singleOpt_summary_df 

In [54]:
# combine the dataframes from all models
combined_singleOpt_summary_df = combine_singleOpt_summary_dataframes()

# Save the data with the below sheetname: 
combined_singleOpt_summary_df.to_excel(WRITER, sheet_name = 'SingleOpt_Summary', index = False)

### Sheet 11. JointOpt
The purpose of this block is to generate the data for sheet: JointOpt

In [55]:
# get per scenario jointOpt dataframe
def get_per_scenario_joinOpt_dataframe(scenario, period_filter):
    
    result_df_path = os.path.abspath(JOINT_OPT_DICT[scenario]['jointOpt_result_df_file'])
    result_df = pd.read_csv(result_df_path)
    result_df.drop(result_df.tail(1).index, inplace = True)
    result_df['WAMM Tactic'] = result_df['Variable'].apply(get_var_description)
    result_df.drop(result_df.columns[0], axis = 1, inplace = True)
    # add filter columns
    start_date_str, end_date_str = JOINT_OPT_DICT[scenario]['jointOpt_start_end_date']
    result_df['Period'] = get_period_value(start_date_str, end_date_str)
    result_df['Time Range'] = get_time_range(start_date_str, end_date_str)
    result_df['Period Filter'] = period_filter
    # reorder the columns
    cols = ['Period Filter','Period', 'Time Range', 'WAMM Tactic'] + result_df.columns.to_list()[:-4]
    result_df = result_df[cols]
    return result_df 

In [56]:
# get all scenarios jointOpt dataframes
def get_all_scenario_jointOpt_dataframes():
    all_scenario_result_ls = []
    period_filter = 1
    for scenario in JOINT_OPT_DICT:
        per_scenario_result = get_per_scenario_joinOpt_dataframe(scenario, period_filter)
        all_scenario_result_ls.append(per_scenario_result)
        period_filter += 1
    
    if all_scenario_result_ls:
        all_scenario_result_df = pd.concat(all_scenario_result_ls)
        return all_scenario_result_df

In [57]:
# get the jointOpt dataframe
all_scenario_result_df = get_all_scenario_jointOpt_dataframes()

# Save the data with the below sheetname: 
all_scenario_result_df.to_excel(WRITER, sheet_name = 'JointOpt', index = False)

### Sheet 12. JointOpt_Summary
The purpose of this block is to generate the data for sheet: JointOpt_Summary

In [58]:
# get jointOpt_summary dataframe
def get_per_scenario_result_contrib_dataframe(scenario):
    result_contrib_path = os.path.abspath(JOINT_OPT_DICT[scenario]['jointOpt_result_contrib_file'])
    result_contrib = pd.read_csv(result_contrib_path)
    result_contrib.drop(result_contrib.columns[[0]], axis = 1, inplace = True)
    # add the columns W1Filter
    result_contrib['W1Filter'] = ['R' + str(i) for i in range(1, len(result_contrib) + 1)]
    return result_contrib

In [59]:
# get optimal and actual spends for each weight
def get_per_scenario_actual_optimal_spend_dataframe(scenario):
    result_df_path = os.path.abspath(JOINT_OPT_DICT[scenario]['jointOpt_result_df_file'])
    result_df = pd.read_csv(result_df_path)
    result_df.drop(result_df.tail(1).index,inplace=True)
    weight_cols = sorted([col for col in result_df.columns if col.startswith('w1')])
    original_sum_cols = sorted([col for col in result_df.columns if col.startswith('Original Sum')])
    solver_sum_cols = sorted([col for col in result_df.columns if col.startswith('Solver Sum')])
    original_sum_ls = result_df[original_sum_cols].sum(axis = 0).values
    solver_sum_ls = result_df[solver_sum_cols].sum(axis = 0).values
    weight_ls = result_df[weight_cols][:1].values[0]
    # create the dataframe for the optimal and actual spend for each weight
    spend_df = pd.DataFrame()
    spend_df['w1'] = weight_ls
    spend_df['Total Actual Spend'] = original_sum_ls
    spend_df['Total Optimal Spend'] = solver_sum_ls
    return spend_df

In [60]:
# add the ASP values, and filter columns 
def add_per_scenario_asp_valus(scenario, jointOpt_df, period_filter):
    start_date_str, end_date_str = JOINT_OPT_DICT[scenario]['jointOpt_start_end_date']
    try:
        stack_temp = df[start_date_str:end_date_str].copy()
        asp_new_customer = stack_temp['PRICE_NEW_CUSTOMER_OMNI'].mean()
        asp_unit = stack_temp['PRICE'].mean()
        jointOpt_df['ASP New Customer'] = asp_new_customer
        jointOpt_df['ASP Unit'] = asp_unit
    except:
        msg = 'Warning! The variables: PRICE_NEW_CUSTOMER_OMNI, PRICE may not exist from the stack dataset!\n'
        Error_msg_ls.append(msg)
        print(msg)
    jointOpt_df['Period'] = get_period_value(start_date_str, end_date_str)
    jointOpt_df['Time Range'] = get_time_range(start_date_str, end_date_str)
    jointOpt_df['Period Filter'] = period_filter
    # reorder the columns
    cols = ['Period Filter','Period', 'Time Range'] + jointOpt_df.columns.to_list()[:-3]
    jointOpt_df = jointOpt_df[cols]
    return jointOpt_df             

In [61]:
# get one scenario jointOpt_summary dataframe
def get_per_scenario_jointOpt_summary_dataframe(scenario, period_filter):
    result_contrib = get_per_scenario_result_contrib_dataframe(scenario)
    spend_df = get_per_scenario_actual_optimal_spend_dataframe(scenario)
    per_scenario_df = pd.merge(result_contrib, spend_df, on='w1')
    jointOpt_df = add_per_scenario_asp_valus(scenario, per_scenario_df, period_filter)
    return jointOpt_df

In [62]:
# get all scenarios jointOpt_summary dataframes
def get_all_scenario_jointOpt_summary_dataframes():
    all_scenario_result_ls = []
    period_filter = 1
    for scenario in JOINT_OPT_DICT:
        per_scenario_result = get_per_scenario_jointOpt_summary_dataframe(scenario, period_filter)
        all_scenario_result_ls.append(per_scenario_result)
        period_filter += 1

    if all_scenario_result_ls:
        all_scenario_result_df = pd.concat(all_scenario_result_ls)
        return all_scenario_result_df

In [63]:
# get the jointOpt_summary dataframe
result_contrib = get_all_scenario_jointOpt_summary_dataframes()

# Save the data with the below sheetname: 
result_contrib.to_excel(WRITER, sheet_name = 'JointOpt_Summary', index = False)

### Sheet 13. S_Curve -- Tested, correct
The purpose of this block is to generate the data for the sheet: S_Curve

In [64]:
# get the list including the file pathes for all media drivers
def get_media_file_list(model):
    
    base_path = MODEL_DICT[model]['output_folder']
    s_curve_start, s_curve_end = MODEL_DICT[model]['S_Curve_start_end_date']
    folder = os.path.join(base_path, 'response_curve_{}_{}'.format(s_curve_start, s_curve_end))
    media_file_list = glob.glob(os.path.join(folder, 'M_*.xlsx')) 
    if media_file_list:
        return media_file_list
    else:
        folder = os.path.join(base_path, 'response_curve*')
        folders = glob.glob(os.path.join(folder))
        s_curve_folder_list = [glob.glob(os.path.join(folder, 'M_*.xlsx')) for folder in folders]
        media_file_list = max(s_curve_folder_list, key=len)
        return media_file_list

In [65]:
# get per_media dataframe
def get_per_media_dataframe(model, media_file): 

    media_name = media_file.split('/')[-1].split('.xlsx')[0]
    media = media_name.rsplit('_', 5)[0]
    # 1. get the dataframe for S_curve
    media_S_curve_df = pd.read_excel(media_file, sheet_name = 's_curve')
    media_S_curve_df = media_S_curve_df[['Raw Variable', 'Contribution']]

    # 2. get the dataframe for the four points
    media_points_df = pd.read_excel(media_file, sheet_name = 'points')
    media_points_df['Flag'] = 'yes'
    media_points_df.rename(columns = {'points' : 'Markup Points',
                                         'x' : 'Raw Variable',
                                         'y' : 'Contribution'}, inplace = True)

    # 3. combine the s_curve and the four_points dataframes
    media_df = media_S_curve_df.append(media_points_df, ignore_index = True)

    # add the columns variable name, variable description for the media
    media_df['Variable Name'] = media
    media_df['Variable Description'] = media_df['Variable Name'].apply(get_var_description).apply(lambda x: x.strip())

    # 4. read time period and add filter columns
    start_end_dates_df = pd.read_excel(media_file, sheet_name = 'modelling')
    start_date_str = str(start_end_dates_df['model_start_date'].values[0])
    end_date_str = str(start_end_dates_df['model_end_date'].values[0])
    media_df['Model Filter'] = MODEL_DICT[model]['model_filter']
    media_df['Model'] = MODEL_DICT[model]['model']
    media_df['Subtitle'] = MODEL_DICT[model]['subtitle']
    media_df['Period'] = get_period_value(start_date_str, end_date_str)
    media_df['Time Range'] = get_time_range(start_date_str, end_date_str)
    media_df['Period Filter'] = 1
    media_df[['Markup Points', 'Flag']] = media_df[['Markup Points', 'Flag']].fillna(0)
    # reorder the columns
    cols = ['Model Filter', 'Model', 'Subtitle', 'Period Filter', 'Period', 'Time Range', 
            'Variable Name', 'Variable Description', 'Raw Variable', 'Contribution', 'Markup Points', 'Flag']
    media_df = media_df[cols]

    return media_df

In [66]:
# get the dataframe for one model, including all media drivers and four points
def get_per_model_S_curve_dataframe(model, media_file_ls):

    per_model_ls = []
    for media_file in media_file_ls:
        media_df = get_per_media_dataframe(model, media_file)
        per_model_ls.append(media_df)

    if per_model_ls:
        S_curve_df = pd.concat(per_model_ls)
        return S_curve_df

In [67]:
# combine the dataframes for all models
def combine_s_curve_dataframes():
    s_curve_ls = []
    for model in MODEL_DICT:
        media_file_ls = get_media_file_list(model)
        s_curve = get_per_model_S_curve_dataframe(model, media_file_ls)
        s_curve_ls.append(s_curve)
    s_curve_df = pd.concat(s_curve_ls)
    
    return s_curve_df

In [68]:
# combine the dataframes from all models
combined_s_curve_df = combine_s_curve_dataframes()

# save the s_curve_df
combined_s_curve_df.to_excel(WRITER, sheet_name = 'S_Curve', index = False)

### 14. Base_Incremental
The purpose of this block is to generate the data required for the base_incremental stacked charts

In [69]:
# get base_incremental dataframe
def get_base_incremental_dataframe(model):
    
    base_path = MODEL_DICT[model]['output_folder']
    base_incremental_path = os.path.join(base_path, 'base_incremental', 'base_incremental.xlsx')
    base_vol_df = pd.read_excel(base_incremental_path, sheet_name = 'base_vol')
    base_incremental_df = pd.read_excel(base_incremental_path,  sheet_name = 'base_incremental')
    for col in base_incremental_df.columns:
        base_incremental_df[col] = base_incremental_df[col].apply(lambda x : x if x > 0 else 0)
    base_incremental_df = base_vol_df.merge(base_incremental_df, left_index = True, right_index = True)
    return base_incremental_df

In [70]:
# transform the base_incremental dataframe by multipling the media driver to a factor, it is using None by default
def transform_base_incremental_dataframe(base_incremental_df, factor = None):
    if not factor:
        return base_incremental_df
    base_value = base_incremental_df['base']
    transformed_base_incremental_df = pd.DataFrame()
    for col in base_incremental_df.columns:
        if col in ['index', 'base']:
            transformed_base_incremental_df[col] = base_incremental_df[col] 
        else:
            transformed_base_incremental_df[col] = base_value + base_incremental_df[col] * factor
            base_value = transformed_base_incremental_df[col]
    return transformed_base_incremental_df

In [71]:
# pivot the base_incremental dataframe after transformation, convert the media variables from different columns into one column
def pivot_base_incremental_dataframe(transformed_base_incremental_df):
    pivot_base_incremental_ls = []
    for col in transformed_base_incremental_df.columns:
        if col == 'index':
            continue
        per_media_df = transformed_base_incremental_df.melt(id_vars = ['index'], value_vars = [col], ignore_index = False)
        pivot_base_incremental_ls.append(per_media_df)
    pivot_base_incremental_df = pd.concat(pivot_base_incremental_ls, ignore_index = True)
    pivot_base_incremental_df.rename(columns = {'variable': 'Attribute', 'value': 'Value'}, inplace = True)
    return pivot_base_incremental_df

In [72]:
# update base_incremental dataframe by adding filter columns
def update_base_incremental_dataframe(model, pivot_base_incremental_df):
    start_date_str = str(pivot_base_incremental_df['index'].min()).split(' ')[0]
    end_date_str = str(pivot_base_incremental_df['index'].max()).split(' ')[0]
    pivot_base_incremental_df['Model Filter'] = MODEL_DICT[model]['model_filter']
    pivot_base_incremental_df['Model'] = MODEL_DICT[model]['model']
    pivot_base_incremental_df['Subtitle'] = MODEL_DICT[model]['subtitle']
    pivot_base_incremental_df['Period'] = get_period_value(start_date_str, end_date_str)
    pivot_base_incremental_df['Time Range'] = get_time_range(start_date_str, end_date_str)
    pivot_base_incremental_df['Period Filter'] = 1
    cols = ['Model Filter', 'Model', 'Subtitle', 'Period Filter', 'Period', 'Time Range', 'index', 'Attribute', 'Value']
    pivot_base_incremental_df = pivot_base_incremental_df[cols]
    return pivot_base_incremental_df

In [73]:
# combine base_incremental dataframe from all models
def combine_base_incremental_dataframes():
    base_incremental_ls = []
    for model in MODEL_DICT:
        base_incremental = get_base_incremental_dataframe(model)
        transformed_base_incremental = transform_base_incremental_dataframe(base_incremental, factor = None)
        pivot_base_incremental = pivot_base_incremental_dataframe(transformed_base_incremental)
        updated_base_incremental = update_base_incremental_dataframe(model, pivot_base_incremental)   
        base_incremental_ls.append(updated_base_incremental)
    base_incremental_df = pd.concat(base_incremental_ls)
    
    return base_incremental_df

In [74]:
# combine the dataframes from all models
combined_base_incremental_df = combine_base_incremental_dataframes()

# save the base_incremental_df
combined_base_incremental_df.to_excel(WRITER, sheet_name = 'Base_Incremental', index = False)

## Export and save the Excel sheets

In [75]:
WRITER.save()

In [76]:
print(round((datetime.now() - start_time).seconds/60, 2), 'mins') 
print(''.join(list(set(Error_msg_ls))))

0.38 mins

